# 96- One-Time Deduplication for Historical Data


This notebook performs a one-time cleanup of historical duplicate records
in key Delta tables using the correct primary keys. It ensures data
integrity for downstream analytics and modeling.

Steps:
- Identify duplicate records
- Drop duplicates based on key columns
- Overwrite the cleaned data in Delta format


## 02 - Ingest GTFS Real-Time Data (Bronze Layer)

In [0]:
from pyspark.sql.functions import col

In [0]:
BRONZE_RT_PATH = "dbfs:/bronze/gtfs_rt/"
df_rt = spark.read.format("delta").load(BRONZE_RT_PATH)

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_rt.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") > 1) \
  .show()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_rt.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_rt= df_rt.dropDuplicates(["vehicle_id", "timestamp"])
display(df_rt)

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_rt.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_rt.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_rt.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .save(BRONZE_RT_PATH)

## 03 - Transform GTFS Real-Time Data (Silver Layer)

In [0]:
SILVER_RT_PATH = "dbfs:/silver/gtfs_rt/"
df_silver= spark.read.format("delta").load(SILVER_RT_PATH)

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_silver.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:
duplicate_keys = df_silver.groupBy("vehicle_id", "timestamp") \
                   .count() \
                   .filter(col("count") > 1) \
                   .select("vehicle_id", "timestamp")
duplicate_rows = df_silver.join(duplicate_keys, on=["vehicle_id", "timestamp"], how="inner").orderBy("vehicle_id", "timestamp")

duplicate_rows.display()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_silver.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_silver= df_silver.dropDuplicates(["vehicle_id", "timestamp"])
display(df_silver)

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_silver.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_silver.groupBy("vehicle_id", "timestamp") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .save(SILVER_RT_PATH)

## 04 - Enrich GTFS Real-Time with Static Data (Gold Layer)

In [0]:
GOLD_PATH = "dbfs:/gold/gtfs_rt_enriched/"
df_enriched= spark.read.format("delta").load(GOLD_PATH)

In [0]:
# Get all column names except "joined_at"
columns_to_check = [c for c in df_enriched.columns if c not in ["joined_at", "processed_at"]]
# Replace `df_rt` with your actual DataFrame name
df_enriched.groupBy(columns_to_check) \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_enriched.groupBy(columns_to_check) \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_enriched= df_enriched.dropDuplicates(columns_to_check)
display(df_enriched)

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_enriched.groupBy(columns_to_check) \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_enriched.groupBy(columns_to_check) \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_enriched.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .save(GOLD_PATH)

## 05 - Ingest Hourly Weather Data from NWS API (Bronze Layer)

In [0]:
BRONZE_WEATHER_PATH = "dbfs:/bronze/weather/"
df_weather= spark.read.format("delta").load(BRONZE_WEATHER_PATH)

In [0]:
# Replace `df_rt` with your actual DataFrame name
df_weather.groupBy("forecast_time") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_weather.groupBy("forecast_time") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
duplicate_keys = df_weather.groupBy("forecast_time") \
                   .count() \
                   .filter(col("count") > 1) \
                   .select("forecast_time")
duplicate_rows = df_weather.join(duplicate_keys, on=["forecast_time"], how="inner")

duplicate_rows.orderBy(col("forecast_time").desc()).display()

In [0]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

w = Window.partitionBy("forecast_time").orderBy(col("forecast_retrieved_at").desc())

df_weather_deduped = (
    df_weather.withColumn("rn", row_number().over(w))
              .filter("rn = 1")
              .drop("rn")
)


In [0]:
# Replace `df_rt` with your actual DataFrame name
df_weather_deduped.groupBy("forecast_time") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_weather_deduped.groupBy("forecast_time") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_weather_deduped.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .save(BRONZE_WEATHER_PATH)

## 06 - Transform Hourly Weather Data (Silver Layer)

In [0]:
SILVER_PATH = "dbfs:/silver/weather/"
df_silver= spark.read.format("delta").load(SILVER_PATH)

In [0]:
# Replace `df_rt` with your actual DataFrame name
df_silver.groupBy("forecast_time") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_silver.groupBy("forecast_time") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
duplicate_keys = df_silver.groupBy("forecast_time") \
                   .count() \
                   .filter(col("count") > 1) \
                   .select("forecast_time")
duplicate_rows = df_silver.join(duplicate_keys, on=["forecast_time"], how="inner")

duplicate_rows.orderBy(col("forecast_time").desc()).display()

In [0]:
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

w = Window.partitionBy("forecast_time").orderBy(col("forecast_retrieved_at").desc())

df_silver_deduped = (
    df_silver.withColumn("rn", row_number().over(w))
              .filter("rn = 1")
              .drop("rn")
)


In [0]:

# Replace `df_rt` with your actual DataFrame name
df_silver_deduped.groupBy("forecast_time") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:

# Replace `df_rt` with your actual DataFrame name
df_silver_deduped.groupBy("forecast_time") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
df_silver_deduped.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .save(SILVER_PATH)

## 07 - Join Transit Data with Weather (Gold Layer)

In [0]:
JOINED_PATH = "dbfs:/gold/gtfs_rt_weather_joined/"
df_joined= spark.read.format("delta").load(JOINED_PATH)

In [0]:
# Replace `df_rt` with your actual DataFrame name
df_joined.groupBy(["vehicle_id", "event_ts", "forecast_time"]) \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:
# Replace `df_rt` with your actual DataFrame name
df_joined.groupBy("vehicle_id", "event_ts", "forecast_time") \
  .count() \
  .filter(col("count") == 1) \
  .count()

In [0]:
duplicate_keys = df_joined.groupBy("vehicle_id", "event_ts", "forecast_time") \
                   .count() \
                   .filter(col("count") > 1) \
                   .select("vehicle_id", "event_ts", "forecast_time")
duplicate_rows = df_joined.join(duplicate_keys, on=["vehicle_id", "event_ts", "forecast_time"], how="inner")

duplicate_rows.orderBy(col("forecast_time").desc(), col("event_ts").desc(), col("event_ts").desc()).display()

In [0]:
df_joined_dedupe= df_joined.dropDuplicates(["vehicle_id", "event_ts", "forecast_time"])
display(df_joined_dedupe)

In [0]:
# Replace `df_rt` with your actual DataFrame name
df_joined_dedupe.groupBy("vehicle_id", "event_ts", "forecast_time") \
  .count() \
  .filter(col("count") > 1) \
  .count()

In [0]:
# Replace `df_rt` with your actual DataFrame name
df_joined_dedupe.groupBy("vehicle_id", "event_ts", "forecast_time") \
  .count() \
  .filter(col("count") == 1) \
  .count()


In [0]:
df_joined_dedupe.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("ingestion_date") \
    .save(JOINED_PATH)